# NeVA Training / Inference Tutorial

### Note:
Currently, this notebook must be run in a NeMo container. An example command to launch the container:

```
docker run --gpus all -it --rm -v <your_nemo_dir>:/opt/NeMo --shm-size=8g \
     -p 8888:8888 --ulimit memlock=-1 --ulimit \
      stack=67108864 <your_nemo_container>
```

## Introduction

This notebook illustrates how to train and perform inference using NeVA with the NeMo Toolkit. NeVA originates from [LLaVA](https://github.com/haotian-liu/LLaVA) (Large Language and Vision Assistant) and is a powerful multimodal image-text instruction tuned model optimized within the NeMo framework. 


This tutorial will guide you through the following topics:
1. Training a NeVA model
2. Performing inference with the trained model

## Datasets

### Pre-Training Dataset

The pre-training dataset is open-sourced from the LLaVA implementation and can be downloaded [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Pretrain). The dataset consists of a 558K subset of the LAION-CC-SBU dataset with BLIP captions. 

### Instruction Tuning Dataset

The instruction tuning annotations are sourced from the LLaVA implementation and are available [here](https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/blob/main/llava_v1_5_mix665k.json)

## Training


### Feature Alignment Pre-Training

An example of a pre-training script:

```
torchrun --nproc_per_node=4 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_pretrain.py \
 ++cluster_type=BCP \
 trainer.precision=bf16 \
 model.megatron_amp_O2=True \
 trainer.num_nodes=1 \
 trainer.devices=4 \
 trainer.val_check_interval=1000 \
 trainer.limit_val_batches=5 \
 trainer.log_every_n_steps=1 \
 trainer.max_steps=1000 \
 model.micro_batch_size=1 \
 model.global_batch_size=2 \
 model.tensor_model_parallel_size=4 \
 model.pipeline_model_parallel_size=1 \
 model.mcore_gpt=False \
 model.transformer_engine=False \
 model.mm_cfg.llm.from_pretrained=null \
 exp_manager.create_checkpoint_callback=True \
 model.data.data_path=/lustre/fsw/coreai_dlalgo_genai/datasets/LLaVA-Pretrain-LCS-558K/blip_laion_cc_sbu_558k.json \
 model.data.image_folder=/lustre/fsw/coreai_dlalgo_genai/datasets/LLaVA-Pretrain-LCS-558K/images \
 model.tokenizer.library=sentencepiece \
 model.tokenizer.model=/lustre/fsw/coreai_dlalgo_genai/datasets/checkpoints/nemotron-3/mt_nlg_plus_multilingual_ja_zh_the_stack_frac_015_256k.model \
 model.encoder_seq_length=4096 \
 model.num_layers=32 \
 model.hidden_size=4096 \
 model.ffn_hidden_size=16384 \
 model.num_attention_heads=32 \
 model.normalization=layernorm1p \
 model.do_layer_norm_weight_decay=False \
 model.apply_query_key_layer_scaling=True \
 model.activation=squared-relu \
 model.headscale=False \
 model.position_embedding_type=rope \
 model.rotary_percentage=0.5 \
 model.num_query_groups=null \
 model.data.num_workers=0 \
 model.mm_cfg.llm.from_pretrained=/lustre/fsw/coreai_dlalgo_genai/datasets/checkpoints/nemotron-3/8B_strict-skua_4200 \
 model.mm_cfg.llm.model_type=nvgpt \
 model.data.conv_template=nvgpt \
 model.mm_cfg.vision_encoder.from_pretrained='openai/clip-vit-large-patch14' \
 model.mm_cfg.vision_encoder.from_hf=True \
 model.data.image_token_len=256 \
 model.optim.name="fused_adam" \
 exp_manager.create_wandb_logger=False \
 exp_manager.wandb_logger_kwargs.project=neva_demo
```

### Image-Language Pair Instruction Fine-Tuning


## Inference


### From Pre-trained Checkpoints

### Running Inference

Once either the necessary checkpoints have been loaded or the training workflow is complete, inference can be executed with the following command:
